In [ ]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import skimage.io
import skimage.filters
import skimage.morphology
from skimage.filters import try_all_threshold
import matplotlib.image as mpimg
from scipy import ndimage as ndi
from skimage import exposure

In [ ]:
# use this when working on desktop
path = '/Desktop/temp_images/NP_penetration/' # The folder where the .tiff files are located
filename = '5p_40nm_brain_1_striatum_100x_z_stack_2'

In [ ]:
# use this when working on laptop
path = '/Desktop/IHC_Data/NP_penetration/'
filename = '5p_100nm_brain_1_striatum_100x_z_stack_2'

In [ ]:
img = skimage.io.imread(os.path.expanduser('~')+path+filename+'.tif')
print(img.shape)
print(img.dtype)

In [ ]:
DAPI = img[0::2]
NP = img[1::2]

print(DAPI.shape)
print(NP.shape)

In [ ]:
def show_plane(ax, plane, cmap="gray", title=None):
    ax.imshow(plane, cmap=cmap)
    ax.axis("off")

    if title:
        ax.set_title(title)


(n_plane, n_row, n_col) = DAPI.shape
_, (a, b, c) = plt.subplots(ncols=3, figsize=(15, 5))

show_plane(a, DAPI[n_plane // 2], title=f'Plane = {n_plane // 2}')
show_plane(b, DAPI[:, 770, :], title=f'Row = {770}')
show_plane(c, DAPI[:, :, n_col // 2], title=f'Column = {n_col // 2}')

In [ ]:
## For now, just using this function to adjust brightness and make the images more viewer friendly

def plot_hist(ax, data, title=None):
    # Helper function for plotting histograms
    ax.hist(data.ravel(), bins=256)
    ax.ticklabel_format(axis="y", style="scientific", scilimits=(0, 0))

    if title:
        ax.set_title(title)


gamma_low_val = 0.5
gamma_low = exposure.adjust_gamma(DAPI, gamma=gamma_low_val)

_, ((a, b), (d, e)) = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

show_plane(a, DAPI[14], title='Original')
show_plane(b, gamma_low[14], title=f'Gamma = {gamma_low_val}')

plot_hist(d, DAPI)
plot_hist(e, gamma_low)

In [ ]:
## This function can be used to look through subsequent steps of the entire volume of the z-stack

def display(im3d, cmap="gray", step=2):
    _, axes = plt.subplots(nrows=5, ncols=5, figsize=(12, 15))

    vmin = im3d.min()
    vmax = im3d.max()

    for ax, image in zip(axes.flatten(), im3d[::step]):
        ax.imshow(image, cmap=cmap, vmin=vmin, vmax=vmax)
        ax.set_xticks([])
        ax.set_yticks([])


display(gamma_low)

In [ ]:
DAPI_intensities = np.sum(np.sum(DAPI, axis=1), axis=1)
NP_intensities = np.sum(np.sum(NP, axis=1), axis=1)
DAPI_intensities.shape
NP_intensities.shape

In [ ]:
z = np.arange(len(DAPI_intensities))

plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh(z, DAPI_intensities, align='center')
ax.set_ylabel('relative z position [um]')
ax.set_xlabel('Sum of DAPI intensities')
ax.set_title('Sum of intensities in the DAPI channel vs. z-position')

plt.show()

In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh(z, NP_intensities, align='center', color='firebrick')
ax.set_ylabel('relative z position [um]')
ax.set_xlabel('Sum of NP intensities')
ax.set_title('Sum of intensities in the NP channel vs. z-position')
#ax.axes.set_xlim(1600000, 3200000)
ax.ticklabel_format(axis='x', style='sci', scilimits=(4,4))

plt.show()

In [ ]:
"""" Currently working on how to decide where we've officially "entered" a z-plane within the slice.
    Definitely not sure if this is a proper way to do it, but I first just want to subtract out the average DAPI
    intensity that exists in the first 4 and last 4 z-planes, where I know we're not in the slice and all signal 
    should be attributed to background."""

subtract_val = np.mean(np.array([np.mean(np.mean(DAPI, axis=1), axis=1)[0:4],np.mean(np.mean(DAPI, axis=1), axis=1)[-4:]]))
DAPI_int_adj = DAPI_intensities-(subtract_val*1024*1024)

In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh(z, DAPI_int_adj, align='center')
ax.set_ylabel('relative z position [um]')
ax.set_xlabel('Sum of adjusted DAPI intensities')
ax.set_title('Sum of adjusted intensities in the DAPI channel vs. z-position')

plt.show()

Now I'm going to use a series of processing steps to determine the number of blobs (in this case cell nuclei) present in each z stack. I've played around in ImageJ to figure out the properties that work best for this particular image. Hopefully it is easily adaptable to other images within the set.

In [ ]:
DAPI.shape

In [ ]:
threshold = 38
DAPI_binary = DAPI >= threshold
DAPI_binary_plot = plt.imshow(1-DAPI_binary[15], cmap='Greys')
plt.show()

In [ ]:
DAPI_count = np.zeros(DAPI.shape[0])


for z in range(0,DAPI.shape[0]):
    dilated = skimage.morphology.binary_dilation(DAPI_binary[z])
    eroded = skimage.morphology.binary_erosion(dilated)
    eroded = ndi.binary_fill_holes(eroded)
    DAPI_clean = skimage.morphology.remove_small_objects(eroded, min_size=50)
    _, DAPI_count[z] = ndi.label(DAPI_clean)

In [ ]:
z = np.arange(len(DAPI_intensities))

def first_nonzero(arr, invalid_val=-1):
    mask = arr!=0
    return np.where(mask.any(), mask.argmax(), invalid_val)

tissue_bottom = 1+first_nonzero(DAPI_count)

plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh(z, DAPI_count, align='center')
ax.set_ylabel('relative z position [um]')
ax.set_xlabel('# of cell nuclei')
ax.set_title('# cell nuclei vs. z-position')

plt.show()
print('The z-stack first enters the slice at the %dth z-plane' % tissue_bottom)

Since we now know the slice in which we enter the tissue, let's now quantify the # of particles at each depth throughout the slice.

In [ ]:
NP.shape

In [ ]:
threshold = 54
NP_binary = NP >= threshold
NP_binary_plot = plt.imshow(1-NP_binary[7], cmap='Greys')
plt.show()

In [ ]:
NP_count = np.zeros(NP.shape[0])


for z in range(0,NP.shape[0]):
    dilated = skimage.morphology.binary_dilation(NP_binary[z])
    eroded = skimage.morphology.binary_erosion(dilated)
    eroded = ndi.binary_fill_holes(eroded)
    NP_clean = skimage.morphology.remove_small_objects(eroded, min_size=10)
    _, NP_count[z] = ndi.label(NP_clean)

In [ ]:
tissue_bottom

In [ ]:
z = np.arange(len(NP_intensities))

plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh(z, NP_count, align='center', color='firebrick')
ax.set_ylabel('relative z position [um]')
ax.set_xlabel('# of NPs in z-plane')
ax.set_title('# NPs vs. z-position')

plt.show()

Now let's use the now defined bottom of the slice to normalize "relative z position" to actual tissue depth

In [ ]:
p_depth = -(z-(tissue_bottom-1))

plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh(p_depth, NP_count, align='center', color='firebrick')
ax.set_ylabel('penetration depth [um]')
ax.set_xlabel('# of NPs')
ax.set_title('NP penetration depth')
ax.set_ylim(min(p_depth),5)

plt.show()

Also want to be able to make a dot plot in GraphPad - so want to export a csv that I can manipulate in excel

In [ ]:
csv_export = []
for depths in range(0,NP_count.size):
    if int(NP_count[depths]) != 0:
        for n in range(0,int(NP_count[depths])):
            csv_export.append(p_depth[depths])

export_df = pd.DataFrame(data={"depths": csv_export})
export_df.to_csv(os.path.expanduser('~')+path+filename+'.csv', sep=',',index=False)

In [ ]:
export_df

In [ ]:
DAPI_clean_plot = plt.imshow(1-DAPI_clean[12], cmap='Greys')
plt.show()

In [ ]:
dilated = skimage.morphology.binary_dilation(DAPI_binary[5])
DAPI_dilated_plot = plt.imshow(1-dilated, cmap='Greys')

plt.show()

In [ ]:
eroded = skimage.morphology.binary_erosion(dilated)
eroded = ndi.binary_fill_holes(eroded)
#DAPI_clean = skimage.morphology.remove_small_objects(eroded, min_size=50)
DAPI_erode_plot = plt.imshow(1-eroded, cmap='Greys')
plt.show()